In [3]:
import mediapipe as mp
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")


In [4]:
#Khởi tạo mediapie
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

#Hàm tính khoảng cách

In [19]:
from math import sqrt

# rescale frame 1/2
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

#Tính khoảng cách giữa 2 điểm trong không gian 2D
def calculate_distance(pointX, pointY) -> float:
    '''
    Calculate a distance between 2 points
    '''

    x1, y1 = pointX
    x2, y2 = pointY

    return sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

#Tính góc giữa 3 điểm trong không gian 3D
def calculate_angle(pointA, pointB, pointC) -> float:
    """
    Tính góc giữa ba điểm trong không gian 3D (đơn vị: độ)
    a, b, c là danh sách chứa tọa độ 3D: [x, y, z]
    """
    A = np.array(pointA)
    B = np.array(pointB)
    C = np.array(pointC)

    # Vector BA và BC
    BA = A - B
    BC = C - B

    # Tính tích vô hướng
    dot_product = np.dot(BA, BC)
    
    # Tính độ dài vector
    norm_BA = np.linalg.norm(BA)
    norm_BC = np.linalg.norm(BC)

    # Tính góc bằng công thức cos(theta) = (A.B) / (|A| * |B|)
    cos_theta = dot_product / (norm_BA * norm_BC)
    
    # Chuyển từ radian sang độ
    angle = np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

    return angle

# def calculate_angle(pointA, pointB, pointC) -> float:
#     '''
#     Tính góc giữa 3 điểm trong mặt phẳng OYZ (bỏ qua tọa độ x).
#     '''
#     # Chỉ lấy tọa độ (y, z) của các điểm
#     A = np.array([pointA[1], pointA[2]])  # (yA, zA)
#     B = np.array([pointB[1], pointB[2]])  # (yB, zB)
#     C = np.array([pointC[1], pointC[2]])  # (yC, zC)

#     # Vector BA và BC trong mặt phẳng OYZ
#     BA = A - B
#     BC = C - B

#     # Tích vô hướng
#     dot_product = np.dot(BA, BC)

#     # Độ dài vector
#     norm_BA = np.linalg.norm(BA)
#     norm_BC = np.linalg.norm(BC)

#     # Tính cos(theta)
#     cos_theta = dot_product / (norm_BA * norm_BC)

#     # Chuyển thành góc độ
#     angle = np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

#     return angle


test

In [6]:
import numpy as np

# Tọa độ có z khác 0
pointA = (1, 0, 1)
pointB = (0, 0, 0)
pointC = (0, 1, 2)

angle = calculate_angle(pointA, pointB, pointC)
print(f"Góc giữa 3 điểm là: {angle:.2f} độ") # 50.77 độ 


Góc giữa 3 điểm là: 50.77 độ


## 4. Xuống quá xâu 
Góc giữa hông, vai, đầu gối nhỏ 


In [7]:
#Chọn các điểm cần lấy
Important_kp = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
]

#Tạo header cho dataframe
header = ["label"]

for kp in Important_kp:
    header.extend([f"{kp}_x", f"{kp}_y", f"{kp}_z", f"{kp}_visibility"])

header

['label',
 'NOSE_x',
 'NOSE_y',
 'NOSE_z',
 'NOSE_visibility',
 'LEFT_SHOULDER_x',
 'LEFT_SHOULDER_y',
 'LEFT_SHOULDER_z',
 'LEFT_SHOULDER_visibility',
 'RIGHT_SHOULDER_x',
 'RIGHT_SHOULDER_y',
 'RIGHT_SHOULDER_z',
 'RIGHT_SHOULDER_visibility',
 'LEFT_HIP_x',
 'LEFT_HIP_y',
 'LEFT_HIP_z',
 'LEFT_HIP_visibility',
 'RIGHT_HIP_x',
 'RIGHT_HIP_y',
 'RIGHT_HIP_z',
 'RIGHT_HIP_visibility',
 'LEFT_KNEE_x',
 'LEFT_KNEE_y',
 'LEFT_KNEE_z',
 'LEFT_KNEE_visibility',
 'RIGHT_KNEE_x',
 'RIGHT_KNEE_y',
 'RIGHT_KNEE_z',
 'RIGHT_KNEE_visibility',
 'LEFT_ANKLE_x',
 'LEFT_ANKLE_y',
 'LEFT_ANKLE_z',
 'LEFT_ANKLE_visibility',
 'RIGHT_ANKLE_x',
 'RIGHT_ANKLE_y',
 'RIGHT_ANKLE_z',
 'RIGHT_ANKLE_visibility']

In [8]:
#Đọc dữ liệu từ frame trả về points
def extract_important_keypoints(results) -> list:
    '''
    Extract important keypoints from mediapipe pose detection
    '''
    landmarks = results.pose_landmarks.landmark

    data = []
    for lm in Important_kp:
        keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
        data.append([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])
    
    return np.array(data).flatten().tolist()

Xem data chuẩn

In [9]:
#Làm sạch dữ liệu dataframe trước khi thêm dữ liệu mới
df = pd.DataFrame(columns=["video", "knee_angle", "hip_angle"])

In [ ]:
#Xử lý từng video để tính khoảng cách chân, đàu gốin 
def process_frame(Video_folder, Video_name):
    
    Cap = cv2.VideoCapture(f"{Video_folder}/{Video_name}")

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while Cap.isOpened():
            ret, image = Cap.read()

            if not ret:
                break

            # Chuyển ảnh sang RGB
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = rescale_frame(image, percent=50)
            image.flags.writeable = True  # Make the image writable

            #trích xuất kpkp
            results = pose.process(image)

            # Kiểm tra có nhận được keypoint không
            if not results.pose_landmarks:
                continue
            
            # Chuyển lại ảnh sang BGR để hiển thị
            # image.flags.writeable = True
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            #khởi tạo biến tính
            knee_angle = None
            hip_angle = None

            
            try:
                # Lấy kp từ frame
                row = extract_important_keypoints(results)
                X = pd.DataFrame([row], columns=header[1:])

                # Tinhs toán và so sánh góc giữa 2 đầu gối
                landmarks = results.pose_landmarks.landmark

                #Chuẩn hóa z theo nose và hip
                # Chọn hai điểm làm chuẩn
                nose_z = landmarks[mp_pose.PoseLandmark.NOSE.value].z
                hip_z = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z

                # Tính khoảng cách tham chiếu
                scale_factor = abs(nose_z - hip_z) if abs(nose_z - hip_z) > 0 else 1

                # Hàm chuẩn hóa Z theo khoảng cách
                def rescale_z(z):
                    return (z - nose_z) / scale_factor

                # Cập nhật lại Z sau khi chuẩn hóa
                for point in mp_pose.PoseLandmark:
                    landmarks[point.value].z = rescale_z(landmarks[point.value].z)

                # Góc đầu gối trái 
                left_knee  = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
                left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

                # Góc đầu gối phải
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z]

                right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)

                #Láy trung bình cộng 2 góc
                knee_angle = (left_knee_angle + right_knee_angle) / 2

                #Góc hông phải
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z]

                right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)

                #Góc hông trái
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]

                left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)

                # # Góc đầu gối trái 
                # left_knee  = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                # left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                # left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                # left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)

                # # Góc đầu gối phải
                # right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                # right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                # right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

                # right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)

                # # Lấy trung bình cộng 2 góc
                # knee_angle = (left_knee_angle + right_knee_angle) / 2

                # # Góc hông phải
                # right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                # right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                # right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]

                # right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)

                # # Góc hông trái
                # left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                # left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                # left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]

                # left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)

                #Lấy trung bình cộng 2 góc
                hip_angle = (left_hip_angle + right_hip_angle) / 2

                # Nền
                cv2.rectangle(image, (0, 0), (500, 60), (245, 117, 16), -1)

                # Display hip angle
                cv2.putText(image, "hip_angle", (15, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(hip_angle, 2)), (10, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display knee angle 
                cv2.putText(image, "knee_angle", (150, 12), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(knee_angle, 2)), (150, 40), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Error: {e}")

            # Draw landmarks and connections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            df.loc[len(df)] = [Video_name, knee_angle, hip_angle]

            # plt.imshow(image)
            # plt.axis("off")  # Ẩn trục tọa độ
            # plt.show()

        Cap.release()

# Đường dẫn thư mục chứa video
# video_folder = "Data/Train/Correct"
# video_files = "20250228_150821.mp4"

# video_folder = "Data/Train/Gap_lung"
# video_files = "20250228_165326.mp4"

# video_folder = "Data/Train/Xuong_qua_xau"
# video_files = "VID_20250228_163102.mp4"

# process_frame(video_folder, video_files)



In [ ]:
import os

# Đường dẫn thư mục chứa video
video_folder = "Data/Train/Correct"
# video_folder = "Data/Train/Xuong_qua_xau"
# video_folder = "Data/Train/Gap_lung"
video_files = [f for f in os.listdir(video_folder) if f.endswith(".mp4")]

df = pd.DataFrame(columns=["video", "knee_angle", "hip_angle"])

for video in video_files:
    process_frame(video_folder, video)

In [70]:
df 

,video,knee_angle,hip_angle
0,20250228_150821.mp4,177.050250,172.416820
1,20250228_150821.mp4,177.111893,172.631752
2,20250228_150821.mp4,177.168940,172.688356
3,20250228_150821.mp4,177.265877,172.775888
4,20250228_150821.mp4,177.394901,172.867495
...,...,...,...
1404,VID_20250228_152348.mp4,179.410235,174.956260
1405,VID_20250228_152348.mp4,179.496347,175.024209
1406,VID_20250228_152348.mp4,179.477161,175.067716
1407,VID_20250228_152348.mp4,179.488279,175.071260


In [71]:
#Tính toán tỉ lệ giữa vai và chân
df["ratio"] = df["knee_angle"] / df["hip_angle"]
df

,video,knee_angle,hip_angle,ratio
0,20250228_150821.mp4,177.050250,172.416820,1.026873
1,20250228_150821.mp4,177.111893,172.631752,1.025952
2,20250228_150821.mp4,177.168940,172.688356,1.025946
3,20250228_150821.mp4,177.265877,172.775888,1.025987
4,20250228_150821.mp4,177.394901,172.867495,1.026190
...,...,...,...,...
1404,VID_20250228_152348.mp4,179.410235,174.956260,1.025458
1405,VID_20250228_152348.mp4,179.496347,175.024209,1.025552
1406,VID_20250228_152348.mp4,179.477161,175.067716,1.025187
1407,VID_20250228_152348.mp4,179.488279,175.071260,1.025230


In [72]:
#Phân tích bộ data
df.describe()


,knee_angle,hip_angle,ratio
count,1409.000000,1409.000000,1409.000000
mean,161.496758,163.709150,0.983847
std,17.503546,11.237073,0.045531
min,122.010400,133.331463,0.864368
25%,146.573824,155.455241,0.953215
50%,168.653562,168.721815,0.999295
75%,176.887713,172.688356,1.021958
max,179.500817,175.380450,1.033505


In [54]:
#test
df_false = df[(df["knee_angle"] < 122)]
df_false 


,video,knee_angle,hip_angle,ratio
0,20250228_162937.mp4,114.221551,121.734974,0.938280
1,20250228_162937.mp4,106.892224,114.510218,0.933473
2,20250228_162937.mp4,101.036225,108.772465,0.928877
3,20250228_162937.mp4,95.384067,103.154558,0.924671
4,20250228_162937.mp4,92.085490,99.815675,0.922555
...,...,...,...,...
1025,VID_20250228_163638.mp4,100.364638,106.149115,0.945506
1026,VID_20250228_163638.mp4,106.547887,112.111990,0.950370
1027,VID_20250228_163638.mp4,111.115537,116.412033,0.954502
1028,VID_20250228_163638.mp4,114.921887,119.995514,0.957718


In [55]:
df_false.describe()

,knee_angle,hip_angle,ratio
count,587.000000,587.000000,587.000000
mean,66.952520,75.985434,0.859566
std,30.736467,30.836746,0.061352
min,27.493294,37.203326,0.736440
25%,38.945380,47.706132,0.817368
50%,60.427389,69.065725,0.871875
75%,95.821093,105.239688,0.910459
max,121.927526,132.853157,0.960798


# Kết luận:
